In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from google.colab import drive

In [2]:
# 1. Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_images_from_folder(folder, size=(128, 128), limit=2000):
    images = []
    filenames = []
    for i, filename in enumerate(tqdm(os.listdir(folder), desc="Loading images")):
        if i >= limit:  # Stop after reaching the specified limit
            break
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path).convert('RGB')  # Ensure the image is in RGB format
            img = img.resize(size, Image.Resampling.LANCZOS)  # Resize the image
            img_array = np.array(img) / 255.0  # Normalize to [0, 1]
            images.append(img_array)
            filenames.append(filename)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images), filenames

In [4]:
# Load and preprocess images
image_folder_path = '/content/drive/MyDrive/dataset/sample image1'
encrypted_image_folder = '/content/drive/MyDrive/dataset/encrypted image'
decrypted_image_folder = '/content/drive/MyDrive/dataset/decrypted image'

In [5]:
# Load original and encrypted images
original_images, original_filenames = load_images_from_folder(image_folder_path, size=(128, 128), limit=500)
encrypted_images, encrypted_filenames = load_images_from_folder(encrypted_image_folder, size=(128, 128), limit=500)
decrypted_images, decrypted_filenames = load_images_from_folder(decrypted_image_folder, size=(128, 128), limit=500)


Loading images:  25%|██▌       | 500/2000 [01:09<03:27,  7.24it/s] 


In [7]:
# One-to-one mapping between original, encrypted, and decrypted images
image_mapping = {}
for i in range(len(original_filenames)):
    image_mapping[original_filenames[i]] = {
        'encrypted': encrypted_filenames[i],
        'decrypted': decrypted_filenames[i]
    }

In [8]:
# Displaying the mapping for verification
for original, mapping in list(image_mapping.items())[:500]:
    print(f"Original: {original} -> Encrypted: {mapping['encrypted']} -> Decrypted: {mapping['decrypted']}")

Original: resized_image_1001.bmp -> Encrypted: encrypted_image_1001.bmp -> Decrypted: decrypted_image_1001.bmp
Original: resized_image_1002.bmp -> Encrypted: encrypted_image_1002.bmp -> Decrypted: decrypted_image_1002.bmp
Original: resized_image_1003.bmp -> Encrypted: encrypted_image_1003.bmp -> Decrypted: decrypted_image_1003.bmp
Original: resized_image_1004.bmp -> Encrypted: encrypted_image_1004.bmp -> Decrypted: decrypted_image_1004.bmp
Original: resized_image_1005.bmp -> Encrypted: encrypted_image_1005.bmp -> Decrypted: decrypted_image_1005.bmp
Original: resized_image_1006.bmp -> Encrypted: encrypted_image_1006.bmp -> Decrypted: decrypted_image_1006.bmp
Original: resized_image_1007.bmp -> Encrypted: encrypted_image_1007.bmp -> Decrypted: decrypted_image_1007.bmp
Original: resized_image_1008.bmp -> Encrypted: encrypted_image_1008.bmp -> Decrypted: decrypted_image_1008.bmp
Original: resized_image_1009.bmp -> Encrypted: encrypted_image_1009.bmp -> Decrypted: decrypted_image_1009.bmp
O

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import losses

# Define the CNN architecture
def create_cnn(input_shape):
    model = models.Sequential([
        layers.Conv2D(8, (3, 3),strides=1, padding="same", activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(16, (3, 3),strides=1, padding="same", activation='relu'),
        layers.Dropout(0.2),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), strides=1, padding="same", activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(3, activation='softmax')  # Three classes (original, encrypted, decrypted)
    ])
    return model

# Model setup
input_shape = (128, 128, 3)
cnn_model = create_cnn(input_shape)

cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 8)         │             224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 16)          │           1,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         524,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 530,579 (2.02 MB)

 Trainable params: 530,579 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
import pandas as pd
from tensorflow.keras import layers, models

In [17]:
# Prepare the table data
layers_data = []
for layer in cnn_model.layers:
    layer_info = {
        "Layer Name": layer.name,
        "Kernel Size": getattr(layer, "kernel_size", "-"),
        "Stride": getattr(layer, "strides", "-"),
        "Padding": getattr(layer, "padding", "-"),
        "Activation Function": getattr(layer, "activation", "-").__name__ if hasattr(layer, "activation") else "-",
        "Normalization Technique": "Dropout ({:.1f})".format(layer.rate) if isinstance(layer, layers.Dropout) else "-",
        "Output Shape": layer.output.shape if layer.output is not None else "-"
    }
    layers_data.append(layer_info)

# Convert to DataFrame
df_layers = pd.DataFrame(layers_data)

# Save to Excel file
excel_path = r'D:\education\image encryption 2024\diagram with tables\cnn_layers_summary.xlsx'  # Specify the path to save the file
df_layers.to_excel(excel_path, index=False)

print(f"Table saved to {excel_path}")

Table saved to D:\education\image encryption 2024\diagram with tables\cnn_layers_summary.xlsx
